In [1]:
import socket

UDP_IP = "188.166.115.7"
UDP_PORT = 7001
MESSAGE = b"TYPE=SUBSCRIPTION_REQUEST"

sock = socket.socket(socket.AF_INET, # Internet
                      socket.SOCK_DGRAM) # UDP

sock.sendto(MESSAGE, (UDP_IP, UDP_PORT))
while True:
    packet = sock.recvfrom(65535)
    print(packet)

(b'TYPE=TRADE|FEEDCODE=SP-FUTURE|SIDE=BID|PRICE=2997.75|VOLUME=329', ('188.166.115.7', 7001))
(b'TYPE=PRICE|FEEDCODE=ESX-FUTURE|BID_PRICE=3162.75|BID_VOLUME=2477|ASK_PRICE=3163|ASK_VOLUME=464', ('188.166.115.7', 7001))
(b'TYPE=TRADE|FEEDCODE=SP-FUTURE|SIDE=ASK|PRICE=2998|VOLUME=281', ('188.166.115.7', 7001))
(b'TYPE=PRICE|FEEDCODE=ESX-FUTURE|BID_PRICE=3163|BID_VOLUME=261|ASK_PRICE=3163.25|ASK_VOLUME=2680', ('188.166.115.7', 7001))
(b'TYPE=TRADE|FEEDCODE=SP-FUTURE|SIDE=ASK|PRICE=2998|VOLUME=418', ('188.166.115.7', 7001))
(b'TYPE=PRICE|FEEDCODE=ESX-FUTURE|BID_PRICE=3163|BID_VOLUME=1652|ASK_PRICE=3163.25|ASK_VOLUME=1289', ('188.166.115.7', 7001))
(b'TYPE=TRADE|FEEDCODE=ESX-FUTURE|SIDE=ASK|PRICE=3163.25|VOLUME=833', ('188.166.115.7', 7001))
(b'TYPE=PRICE|FEEDCODE=SP-FUTURE|BID_PRICE=2998.25|BID_VOLUME=1242|ASK_PRICE=2998.5|ASK_VOLUME=1560', ('188.166.115.7', 7001))
(b'TYPE=TRADE|FEEDCODE=ESX-FUTURE|SIDE=BID|PRICE=3163|VOLUME=1505', ('188.166.115.7', 7001))
(b'TYPE=PRICE|FEEDCODE=SP-FUTURE|

KeyboardInterrupt: 